In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# pip install imblearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")

In [2]:
t1=pd.read_csv('D:\Verizon\Version_2\modeling\merged_data\hier_data.csv')
t2=pd.read_csv('D:\Verizon\Version_2\modeling\merged_data\hier_data_micro.csv')
df=pd.concat([t1,t2])
df=df.dropna()

In [3]:
df['action']=0
df['action']=np.where(df['dc_perc'] <= 0.50, 0, 1)
df['action'].value_counts()

0    365877
1     29569
Name: action, dtype: int64

In [4]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
enode_enc = pd.DataFrame(df, columns=['key_ramesh'])
day_enc=pd.DataFrame(df, columns=['day'])
enode_enc['key_enc'] = labelencoder.fit_transform(enode_enc['key_ramesh'])
day_enc['day_enc']=labelencoder.fit_transform(day_enc['day'])
df['enode_enc']=enode_enc['key_enc']
df['day_enc']=day_enc['day_enc']

In [5]:
height_enc = pd.DataFrame(df, columns=['centerline_type'])
height_enc['height_enc'] = labelencoder.fit_transform(height_enc['centerline_type'])
df['height_enc']=height_enc['height_enc']

In [6]:
freq_enc = pd.DataFrame(df, columns=['key_frequency'])
freq_enc['freq_enc'] = labelencoder.fit_transform(freq_enc['key_frequency'])
df['freq_enc']=freq_enc['freq_enc']

In [7]:
df[['key_frequency','freq_enc']].drop_duplicates()

,key_frequency,freq_enc
0,700_5230,0
1,AWS_2050,2
2,PCS_900,5
12603,AWS3_66961,1
98554,PCS_875,4
155999,Cellular_2585,3


In [13]:
df.columns

Index(['day', 'hr', 'enodeb', 'eutrancell', 'carrier_x',
       'sip_sc_abnormal_releases', 'sip_sc_call_answers',
       'sip_sc_call_attempts', 'sip_sc_call_completions', 'sip_sc_call_drops',
       'sip_sc_call_drops_including_handover', 'sip_sc_call_setup_completes',
       'sip_sc_call_setup_failures', 'adjusted_sip_sc_dcs',
       'adjusted_sip_sc_dc_pct', 'handover_attempts', 'handoverfailures',
       'handover_failure_pct', 'qci1_bd_pct_num', 'qci1_bd_pct_den',
       'key_ramesh', 'centerline_type', 'lat_new', 'long_new', 'key_frequency',
       'dc_perc', 'loc_id', 'enode_b', 'summary', 'icon', 'precipIntensity',
       'precipProbability', 'temperature', 'apparentTemperature', 'dewPoint',
       'humidity', 'pressure', 'windSpeed', 'windGust', 'windBearing',
       'cloudCover', 'uvIndex', 'visibility', 'ozone', 'precipType', 'action',
       'enode_enc', 'day_enc', 'height_enc', 'freq_enc'],
      dtype='object')

In [14]:
print(df.shape)
df=df.dropna()

(395446, 50)


In [15]:
X=df[['day_enc','enode_enc','height_enc','freq_enc','hr','pressure','temperature','precipIntensity','humidity', 'windSpeed', 'windBearing','cloudCover', 'uvIndex']]

In [16]:
Y=df[['action']]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [18]:
logreg = LogisticRegression()

In [19]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(276812, 13)
(118634, 13)
(276812, 1)
(118634, 1)


## Logistic

In [21]:
import statsmodels.api as sm
logit_model=sm.Logit(y_train,X_train)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.237131
         Iterations 8
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.109      
Dependent Variable: action           AIC:              131307.4767
Date:               2020-08-09 22:13 BIC:              131444.3809
No. Observations:   276812           Log-Likelihood:   -65641.    
Df Model:           12               LL-Null:          -73668.    
Df Residuals:       276799           LLR p-value:      0.0000     
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     8.0000                                        
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
day_enc           -0.0002   0.0001  -2.9974 0.0027 -0.0004 -0.0001
enode_enc          0.0078   0.0005  15.3185 0.0000  0

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.93


In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[109798      0]
 [  8836      0]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96    109798
           1       0.00      0.00      0.00      8836

    accuracy                           0.93    118634
   macro avg       0.46      0.50      0.48    118634
weighted avg       0.86      0.93      0.89    118634



# Random forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
model.fit(X_train, y_train)

RandomForestClassifier(max_features='sqrt')

In [27]:
rf_predictions = model.predict(X_test)
rf_probs = model.predict_proba(X_test)[:, 1]

Accuracy of logistic regression classifier on test set: 0.93


In [30]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, rf_predictions)))
confusion_matrix = confusion_matrix(y_test,rf_predictions)
print(confusion_matrix)
print(classification_report(y_test, rf_predictions))

Accuracy of logistic regression classifier on test set: 0.92
[[108069   1729]
 [  8228    608]]
              precision    recall  f1-score   support

           0       0.93      0.98      0.96    109798
           1       0.26      0.07      0.11      8836

    accuracy                           0.92    118634
   macro avg       0.59      0.53      0.53    118634
weighted avg       0.88      0.92      0.89    118634



# Boosting

In [32]:
clf = AdaBoostClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))

Accuracy of logistic regression classifier on test set: 0.93


In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test,y_pred)
print(confusion_matrix)

[[109797      1]
 [  8836      0]]


# Gradient boosting

In [34]:
clf = GradientBoostingClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))

Accuracy of logistic regression classifier on test set: 0.93


In [35]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test,y_pred)
print(confusion_matrix)

0.9255525397440869
[[109796      2]
 [  8830      6]]


# Decision Tree

In [36]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))

Accuracy of logistic regression classifier on test set: 0.88


In [37]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test,y_pred)
print(confusion_matrix)

0.8762243538951734
[[101905   7893]
 [  6791   2045]]


# SMOTE

In [38]:
os = SMOTE(random_state = 33)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
os_data_X,os_data_y=os.fit_sample(X_train, y_train)

In [40]:
os_data_y['action'].value_counts()

1    256079
0    256079
Name: action, dtype: int64

In [41]:
X=os_data_X
Y=os_data_y

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [43]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
model.fit(X_train, y_train)

RandomForestClassifier(max_features='sqrt')

In [44]:
pred_train=model.predict(X_train)
pred_test = model.predict(X_test)
rf_probs = model.predict_proba(X_test)[:, 1]

In [45]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test,pred_test)
print(confusion_matrix)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred_test))

[[72057  4836]
 [ 4064 72691]]
0.9420753931063209


In [46]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(pred_train.shape)
print(pred_test.shape)

(358510, 13)
(153648, 13)
(358510, 1)
(153648, 1)
(358510,)
(153648,)


In [465]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances
def r2(model, X_train, y_train):
    return r2_score(y_train, model.predict(X_train))
perm_imp_rfpimp = permutation_importances(model, X_train, y_train, r2)

In [466]:
perm_imp_rfpimp

,Importance
Feature,
enode_enc,0.995252
freq_enc,0.577630
uvIndex,0.515227
precipIntensity,0.379220
hr,0.360819
height_enc,0.236812
day_enc,0.169609
humidity,0.136807
cloudCover,0.079204


In [467]:
X_train['action']=y_train
X_train['prediction']=pred_train
X_test['action']=y_test
X_test['prediction']=pred_test
final=pd.concat([X_train,X_test])

In [468]:
final_mer=pd.merge(final,df[['day_enc','hr','enode_enc','dc_perc']],how='left',left_on=['day_enc','hr','enode_enc'],right_on = ['day_enc','hr','enode_enc'])
X_test_1=pd.merge(X_test,df[['day_enc','hr','enode_enc','dc_perc']],how='left',left_on=['day_enc','hr','enode_enc'],right_on = ['day_enc','hr','enode_enc'])


In [469]:
final_mer=final_mer.dropna()
# final_mer.isna().sum()
X_test_1=X_test_1.dropna()
# X_test_1.isna().sum()

## RF Regressor

In [470]:
 part_1= final_mer[(final_mer.prediction == 0)]

In [471]:
 part_2=final_mer[(final.prediction == 1)]

In [472]:
X_reg=part_2[['day_enc','enode_enc','height_enc','freq_enc','hr','pressure','temperature','precipIntensity','humidity', 'windSpeed', 'windBearing','cloudCover', 'uvIndex']]

In [473]:
Y_reg=part_2[['dc_perc']]

In [474]:
X_train, X_test, y_train, y_test = train_test_split(X_reg, Y_reg, test_size=0.3, random_state=0)

In [475]:
regressor = RandomForestRegressor(n_estimators=100, random_state=0)
regressor.fit(X_train,y_train)

RandomForestRegressor(random_state=0)

In [476]:
y_pred_train = regressor.predict(X_train)
y_pred_test = regressor.predict(X_test)

In [383]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
print(sqrt(mean_squared_error(y_train, y_pred_train)))
print(sqrt(mean_squared_error(y_test, y_pred_test)))

print(mean_absolute_error(y_train, y_pred_train))
print(mean_absolute_error(y_test, y_pred_test))

0.992219214619256
2.385911668394689
0.21954046106685338
0.5827453975176753


In [485]:
from sklearn.metrics import r2_score
print(r2_score(y_train, y_pred_train,multioutput='uniform_average'))
print(r2_score(y_test, y_pred_test,multioutput='uniform_average'))

0.8868183381480732
0.2216469064085299


In [384]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(regressor, X_train, y_train):
    return r2_score(y_train, regressor.predict(X_train))

perm_imp_rfpimp = permutation_importances(regressor, X_train, y_train, r2)

In [385]:
perm_imp_rfpimp

,Importance
Feature,
hr,0.670655
height_enc,0.506649
enode_enc,0.494807
day_enc,0.404739
uvIndex,0.363329
temperature,0.338578
pressure,0.268925
freq_enc,0.215728
cloudCover,0.195875


In [386]:
X_train['action']=y_train
X_train['prediction']=y_pred_train
X_test['action']=y_test
X_test['prediction']=y_pred_test

In [387]:
final_3=pd.concat([X_train,X_test])
final_3=final_3.sort_index(axis=1, level=None, ascending=True)
final_3
del part_1['dc_perc']
part_1=part_1.sort_index(axis=1, level=None, ascending=True)
part_1

,action,cloudCover,day_enc,enode_enc,freq_enc,height_enc,hr,humidity,precipIntensity,prediction,pressure,temperature,uvIndex,windBearing,windSpeed
0,0,0.00,342,4,0,1,3,0.81,0.0004,0,1019.2,65.45,0.0,129.0,6.81
1,0,0.24,84,12,0,1,1,0.92,0.0008,0,1017.3,65.62,0.0,290.0,4.01
3,0,0.01,65,21,2,1,14,0.45,0.0021,0,1024.5,70.78,6.0,55.0,3.47
4,0,0.17,22,25,2,0,8,0.74,0.0000,0,1024.2,64.75,1.0,122.0,10.84
6,0,0.87,317,33,2,0,10,0.77,0.0000,0,1019.9,70.37,4.0,27.0,4.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512148,0,0.03,137,7,1,1,13,0.47,0.0015,0,1016.2,85.81,9.0,20.0,4.73
512149,0,0.58,69,32,0,0,9,0.86,0.0010,0,1023.6,73.65,4.0,245.0,1.77
512152,0,0.31,81,35,2,0,13,0.28,0.0000,0,1022.8,77.19,7.0,169.0,3.63
512154,0,0.31,196,0,0,1,4,0.82,0.0033,0,1018.5,79.39,0.0,125.0,7.42


In [293]:
final_data=pd.concat([final_3,part_1])

In [294]:
# final_data.to_csv ('D:\Verizon\Version_2\modeling\hadoop\\hier_mod_mac_mic.csv', index_label='Index')

In [296]:
final_data.shape

(503620, 20)